<img src="../img/Signet_FNW_1.svg" alt="OVGU_FNW_Logo" width="300" align="right">

# 2.9. Geometrical optics: Refection &amp; refraction

Nowadays we know that light has particle and wave properties, a concept known as *wave-particle duality*. 
However, when light interacts with objects much larger than its tiny wavelength, its wave nature becomes less significant. 
In such scenarios, we can effectively approximate light as traveling in straight lines called rays. 
Geometric optics is this simplified model that allows us to understand *reflection &amp; refraction* as well as (in the next chapter) lenses and optical instruments.

## 2.9.1 Ray model of light

The **ray model of light** assumes that *light travels in straight-line path*, the so-called *light rays*.
Each ray is assumed to be an extremely narrow beam of light.

This assumption of light moving in straight lines, is how we perceive and interpret our surrounding in daily life.
While this assumption is reasonable in many circumstance it also gives rise to a number of interesting effects (see later).

We will use the ray model to explain *reflection* and *refraction*.
In a subsequent chapter, the wave aspect of light will be investigated to understand *interference*, *polarization* and *diffraction*.


## 2.9.2 Reflection 

When light reaches a surface, it is either *reflected*, *absorbed* (transformed to thermal energy), and *transmitted* (if surface is not opaque).
*Mirrors* are designed to reflect most of the light that reaches them.
A single ray of light reaching a plane mirror will be reflected.
The angle at which the ray will be reflected can be found by the following steps:
1. find the *normal perpendicular to the surface*
2. find the *angle of incidence* $\theta_i$, defined as the angle between the normal and the incident ray
3. the *angle of reflection* $\theta_r$, defined as the angle between the normal and the reflected ray, is equal to the angle of incidence

Thus, the **law of reflection** states, **the angle of incidence and the angle of reflection are equal $\theta_i = \theta_r$**.

This concept can be extended to *non-planar surfaces*.
For each ray the normal is found individually.
As most surfaces in daily life are on a microscopical scale non-planar, the will *reflect light into many direction*.
This is called **diffuse reflection** and the reason why the can see objects from various orientation.

In contrast, **specular reflection** reflects an array of parallel rays all at the same reflection angle ("Speculum" is Latin for mirror).
Thus, an object is only visible if our eyes are a the right position w.r.t. mirror to "catch" the reflected rays.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Checkbox

# Global constants
MIRROR_LENGTH = 6
X_MIN, X_MAX = -3, 3
Y_MIN, Y_MAX = -3, 3
NUM_RAYS_MIN, NUM_RAYS_MAX = 1, 10
MIRROR_ANGLE_MIN, MIRROR_ANGLE_MAX = 30, 80
SHIFT_MIN, SHIFT_MAX = -MIRROR_LENGTH / 4, MIRROR_LENGTH / 4

def reflect_ray(incident_angle, normal_angle):
    return 2 * normal_angle - incident_angle

def compute_normal(mirror_angle, x=None, bumpy=False):
    """Computes the local normal angle for the mirror."""
    base_normal = np.deg2rad(mirror_angle) + np.pi / 2  # Base normal for a flat mirror

    if not bumpy or x is None:
        return base_normal  # Flat mirror case

    # Compute local tangent deviation
    bump_slope = 5 * 0.05 * np.cos(5 * x)  # Derivative of 0.05 * sin(5x)
    tangent_angle = np.arctan(bump_slope)  # Local tangent angle due to bump

    # Adjust normal by the tangent deviation
    return base_normal + tangent_angle

def plot_reflection(num_rays, mirror_angle, ray_shift, bumpy_mirror):
    fig, ax = plt.subplots(figsize=(5, 5))  # Set equal aspect ratio
    
    # Mirror setup
    mirror_x = np.linspace(-MIRROR_LENGTH / 2, MIRROR_LENGTH / 2, 1000)
    mirror_y = np.tan(np.deg2rad(mirror_angle)) * mirror_x
    if bumpy_mirror:
        mirror_y += 0.05 * np.sin(5 * mirror_x)
    
    # Define incoming rays
    num_rays = int(num_rays)
    x_start = np.full(num_rays, X_MIN)
    if num_rays == 1:
        y_start = [ray_shift]
    else:
        y_start = np.linspace(ray_shift - 0.1 * MIRROR_LENGTH, ray_shift + 0.1 * MIRROR_LENGTH, num_rays)  # Centered at ray_shift, max +/- 10% of mirror width
    
    for x0, y0 in zip(x_start, y_start):
        # Find intersection with the mirror
        distances = np.abs(mirror_y - y0)
        idx = np.argmin(distances)  # Closest intersection point
        x_intersect, y_intersect = mirror_x[idx], mirror_y[idx]
        
        # Compute normal angle at the intersection
        normal_angle = compute_normal(mirror_angle, x_intersect, bumpy=bumpy_mirror)
        
        # Compute incident and reflected angles
        incident_angle = np.arctan2(y0 - y_intersect, x0 - x_intersect)
        reflected_angle = reflect_ray(incident_angle, normal_angle)
        
        # Incident ray
        ax.plot([x0, x_intersect], [y0, y_intersect], 'm-')
        
        # Reflected ray
        x_reflect_end = x_intersect + MIRROR_LENGTH * np.cos(reflected_angle)
        y_reflect_end = y_intersect + MIRROR_LENGTH * np.sin(reflected_angle)
        ax.plot([x_intersect, x_reflect_end], [y_intersect, y_reflect_end], 'm-')

        #Incident arrow (further away)
        arrow_x_inc = x0 + 0.2 * (x_intersect - x0) # now calculated from the start of the line
        arrow_y_inc = y0 + 0.2 * (y_intersect - y0) # now calculated from the start of the line
        ax.arrow(arrow_x_inc, arrow_y_inc, 0.1*(x_intersect - x0), 0.1*(y_intersect - y0), head_width=0.1, head_length=0.1, fc='m', ec='m')
        
        #Reflected arrow
        arrow_x_ref = x_intersect + 0.2 * (x_reflect_end - x_intersect)
        arrow_y_ref = y_intersect + 0.2 * (y_reflect_end - y_intersect)
        ax.arrow(arrow_x_ref, arrow_y_ref, 0.1*(x_reflect_end - x_intersect), 0.1*(y_reflect_end - y_intersect), head_width=0.1, head_length=0.1, fc='m', ec='m')
        
        # Normal line (dotted, length 0.5)
        x_normal_end = x_intersect + 0.5 * np.cos(normal_angle)
        y_normal_end = y_intersect + 0.5 * np.sin(normal_angle)
        x_normal_start = x_intersect - 0.5 * np.cos(normal_angle)
        y_normal_start = y_intersect - 0.5 * np.sin(normal_angle)
        ax.plot([x_normal_start, x_normal_end], [y_normal_start, y_normal_end], 'k--')
    
    # Plot mirror
    ax.plot(mirror_x, mirror_y, 'k', linewidth=5)
    
    # Format plot
    ax.set_xlim(X_MIN, X_MAX)
    ax.set_ylim(Y_MIN, Y_MAX)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_aspect('equal')  # Ensure equal aspect ratio
    ax.set_title("Reflection at a Rotating Mirror")
    plt.show()

interact(plot_reflection, 
         num_rays=IntSlider(value=1, min=NUM_RAYS_MIN, max=NUM_RAYS_MAX, step=1, description='Number of Rays'),
         mirror_angle=IntSlider(value=MIRROR_ANGLE_MAX, min=MIRROR_ANGLE_MIN, max=MIRROR_ANGLE_MAX, step=1, description='Mirror Angle'),
         ray_shift=FloatSlider(value=0, min=SHIFT_MIN, max=SHIFT_MAX, step=0.1, description='Ray Shift'),
         bumpy_mirror=Checkbox(value=False, description='Bumpy Mirror'))

interactive(children=(IntSlider(value=1, description='Number of Rays', max=10, min=1), IntSlider(value=80, des…

<function __main__.plot_reflection(num_rays, mirror_angle, ray_shift, bumpy_mirror)>

## 2.9.3 Image formation at plane mirrors: Real vs. virtual images

Plane mirror are common in daily life, yet they are somewhat bizarre.
Things appear to be behind the mirror's surface and directions behave strangely: up and down are the same, but left and right are swapped.

To understand this, lets consider an object in front of a plane mirror.
Let's consider two points of the object and two rays come from each point.
The four rays are reflected at the mirror (obeying $\theta_i = \theta_r$) and reach our eye.
Obviously the real object is in front of mirror, but we perceive is as being behind/inside the mirror.
Interestingly, the distance of the object and mirrored image to the mirror appear to be the same.
We call these distances **object distance $d_o$** (distance object to mirror, measure perpendicular to the mirror) and **image distance $d_i$** (distance image to mirror, measure perpendicular to the mirror) and they are **equal $d_o=d_i$** (true only for plane mirror).
Further, the object's height is the same as the image's height.

How does this work?
Let's do a ray reconstruction (see simulation below). 
Beyond the incident and reflecting rays we have to extend the reflecting rays "behind" the mirror.
Where these extended rays intersect, a **image point** is formed.
The image points for all points of the object appear at the same distance as the object is positioned from the mirror, generating the mirrored object.
We know that there is no real object inside the mirror.
Therefore, we call this image an **virtual image**, i.e. the rays do not cross there, only there extension. 
In contrast, **real images** are generated in rays intersect. 
In other words, we could hold a piece of paper where the real image is produced a see a projection. 
This would not work for virtual images as the rays do not intersect.
Our brains are "wired" to interpret diverging rays as images, regardless if there come from an virtual or real source.

Another way to differentiate between virtual and real images is by considering their location relative to the optical instrument (i.e. lenses, see next chapter). 
For an object positioned off-center with respect to a relevant axis, a real image typically forms on the opposite side, while a virtual image appears on the same side.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Constants
MIRROR_X = 0  # Vertical mirror at x = 0
OBJ_Y = -2  # Fixed object height
EYE_Y = -OBJ_Y  # Observer's y position
X_MIN, X_MAX = -4, 4
Y_MIN, Y_MAX = -3, 3

# Global variable for divergence (angle in radians)
DIVERGENCE_ANGLE = 0.1  # Fixed divergence angle
EYE_RADIUS = 0.75 / 2  # fixed eye radius

def plot_image_formation(object_distance):
    # positions of object, eye, and image
    object_x = -object_distance  # Object on the left
    eye_x = -object_distance
    image_x = object_distance  # Virtual image on the right
    image_y = OBJ_Y
        
    # Compute intersection points on the mirror for two rays
    mirror_hit_y_center = (OBJ_Y + EYE_Y) / 2
    
    # Calculate the vertical offset based on divergence angle and object distance
    vertical_offset = object_distance * np.tan(DIVERGENCE_ANGLE / 2)
    
    mirror_hit_y1 = mirror_hit_y_center + vertical_offset  # First ray slightly above center
    mirror_hit_y2 = mirror_hit_y_center - vertical_offset  # Second ray slightly below center
    mirror_hit_x = MIRROR_X
    
    # Calculate correct reflection angles for two rays
    incident_angle1 = np.arctan2(OBJ_Y - mirror_hit_y1, mirror_hit_x - object_distance)  # Angle of first incident ray
    reflected_angle1 = -incident_angle1  # Reflection law: θi = θr
    
    incident_angle2 = np.arctan2(OBJ_Y - mirror_hit_y2, mirror_hit_x - object_distance)  # Angle of second incident ray
    reflected_angle2 = -incident_angle2  # Reflection law: θi = θr
    
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # compute observer position (as a circle)
    reflected_x_end1 = eye_x  # Ensuring ray reaches observer
    reflected_y_end1 = mirror_hit_y1 + np.tan(reflected_angle1) * (eye_x - MIRROR_X)
    
    reflected_x_end2 = eye_x  # Ensuring ray reaches observer
    reflected_y_end2 = mirror_hit_y2 + np.tan(reflected_angle2) * (eye_x - MIRROR_X)
    
    eye_center_y = (reflected_y_end1 + reflected_y_end2) / 2
    
    # Plot object
    ax.plot(object_x, OBJ_Y, 'b^', markersize=10, label="Object")
    ax.text(object_x - 0.2, OBJ_Y - 0.4, 'Object', color='b')
    
    # Plot virtual image
    ax.plot(image_x, image_y, 'r^', markersize=10, label="Virtual Image", alpha=0.5)
    ax.text(image_x + 0.2, image_y - 0.2, 'Virtual image', color='r')

    # First Ray from object to mirror
    ax.plot([object_x, mirror_hit_x], [OBJ_Y, mirror_hit_y1], 'm-', label="Incident Ray 1")
    ax.arrow(object_x + 0.3 * (mirror_hit_x - object_x), OBJ_Y + 0.3 * (mirror_hit_y1 - OBJ_Y),
             0.1 * (mirror_hit_x - object_x), 0.1 * (mirror_hit_y1 - OBJ_Y), head_width=0.1, head_length=0.1, fc='m', ec='m')
    
    # First Reflected ray
    ax.plot([mirror_hit_x, reflected_x_end1], [mirror_hit_y1, reflected_y_end1], 'm-', label="Reflected Ray 1")
    ax.arrow(mirror_hit_x + 0.3 * (reflected_x_end1 - mirror_hit_x), mirror_hit_y1 + 0.3 * (reflected_y_end1 - mirror_hit_y1),
             0.1 * (reflected_x_end1 - mirror_hit_x), 0.1 * (reflected_y_end1 - mirror_hit_y1), head_width=0.1, head_length=0.1, fc='m', ec='m')
    
    # Virtual ray extending from mirror to image for first ray
    ax.plot([mirror_hit_x, image_x], [mirror_hit_y1, image_y], 'r--', label="Virtual Ray 1", alpha=0.5)

    # Second Ray from object to mirror
    ax.plot([object_x, mirror_hit_x], [OBJ_Y, mirror_hit_y2], 'c-', label="Incident Ray 2")
    ax.arrow(object_x + 0.3 * (mirror_hit_x - object_x), OBJ_Y + 0.3 * (mirror_hit_y2 - OBJ_Y),
             0.1 * (mirror_hit_x - object_x), 0.1 * (mirror_hit_y2 - OBJ_Y), head_width=0.1, head_length=0.1, fc='c', ec='c')
    
    # Second Reflected ray
    ax.plot([mirror_hit_x, reflected_x_end2], [mirror_hit_y2, reflected_y_end2], 'c-', label="Reflected Ray 2")
    ax.arrow(mirror_hit_x + 0.3 * (reflected_x_end2 - mirror_hit_x), mirror_hit_y2 + 0.3 * (reflected_y_end2 - mirror_hit_y2),
             0.1 * (reflected_x_end2 - mirror_hit_x), 0.1 * (reflected_y_end2 - mirror_hit_y2), head_width=0.1, head_length=0.1, fc='c', ec='c')
    
    # Virtual ray extending from mirror to image for second ray
    ax.plot([mirror_hit_x, image_x], [mirror_hit_y2, image_y], 'r--', label="Virtual Ray 2", alpha=0.5)
    
    # Plot observer's eye
    circle = plt.Circle((eye_x, eye_center_y), EYE_RADIUS, color='g', fill=True, linewidth=1, label="Observer", zorder=3)
    ax.add_artist(circle)
    ax.text(eye_x, eye_center_y + EYE_RADIUS + 0.2, 'eye', ha='center', va='bottom', color='g', zorder=3)
    
    # Plot mirror
    ax.plot([MIRROR_X, MIRROR_X], [Y_MIN, Y_MAX], 'k', linewidth=5, label="Mirror", zorder=1)
    
    # Formatting
    ax.set_xlim(X_MIN, X_MAX)
    ax.set_ylim(Y_MIN, Y_MAX)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_aspect('equal')
    ax.set_title("Image formation with a plane mirror (two rays)")
    #ax.legend()
    plt.show()

# Interactive widget
interact(plot_image_formation, 
         object_distance=FloatSlider(value=1.5, min=0.5, max=3, step=0.1, description='Object Distance'))

interactive(children=(FloatSlider(value=1.5, description='Object Distance', max=3.0, min=0.5), Output()), _dom…

<function __main__.plot_image_formation(object_distance)>

## 2.9.4 Image formation at curved mirrors

Curved mirrors are typically spherical and can be differentiated as:
* **convex**: surface bulged towards viewer; extend the field of view; e.g. rear view mirrors
* **concave**: surface bulged inwards (like a cave), magnifying mirrors; e.g. shaving/cosmetics mirrors

Mirrors (and lenses) have **focal points** and a **focal length**.
To define these entities, we need incoming rays parallel to the **principle axis**.
The principle axis is defined as the straight line perpendicular to the spherical surface a the center of the mirror.
By considering an *object infinitely far away* from the mirror (e.g. the Sun), we *obtain parallel rays*.
In case of a concave mirror, these incident parallel rays will be reflected and all reflected rays intersect at a single point, the so-called **focal point F**.
In other words, the focal point is the image point of an object positioned infinitely away from the mirror.
For an convex mirror, not the reflected rays but their extension will intersect in the focal point (virtual point).
The focal point is positioned on the principle axis and its distance to the mirror along the principle axis, is the so-called **focal length f**.
Interestingly, for spherical mirrors the radius $r$ of the curvature is twice the focal length $f$:
$$f = \frac{r}{2} \quad \leftrightarrow \quad r = 2 f$$

Strictly speaking, this is only true if the spherical mirror is small compared to its curvature radius, i.e. the reflected rays have only a small angle w.r.t. principle axis, i.e. **paraxial rays** (very useful assumption in geometric optics).
The reason for this is **spherical aberration**. 
Spherical aberration (discussed more for lenses in the next chapter) is an imaging artifact/imperfection and causes the reflecting rays to not perfectly intersect in a single point but rather in a blurry, less focused region.
**Parabolic reflectors** show no spherical aberration but are more challenging, thus expansive, to make.
However, for the remainder of this chapter, we will neglect spherical aberration.

#### Image formation via ray tracing
We know that an object infinitely far away creates an image at the focal point (real for concave and virtual for convex mirrors), but what if the object is considerably closer (but still $\geq f=\frac{r}{2}$ away from the mirror)?
To construct the image in this case, we need at least 2 of these 3 rays and their intersection is where the image is generated:
* **parallel ray**: ray parallel to the primary axis
* **focal point ray**: ray going through the focal point at distance $f$
* **central point ray**: ray going through the central point at distance $r$

For **concave mirrors**, the parallel ray will be reflected by the mirror and becomes a focal point ray.
The focal point ray will be reflected by the mirror and become a parallel ray.
Already the intersection of these two rays tells us where the image form.
The third ray, i.e. central point ray, is by definition perpendicular to the mirror (ray along a imaginary radius line).
Thus, the ray will be reflected perpendicular from the mirror and intersecting with the other two rays at the image point.

For **convex mirrors**, the same rays are used but they must be extended "behind" the mirror.
The intersection of the extended rays shows were the virtual image is formed.

**Note:** convex mirrors (and lenses) always produce virtual images, regardless of the focal length or the position of the object.

#### Mirror equation
We define the **object distance** $d_o$ as the distance between the mirror and the object along the principal axis.
Analogously, we define the **image distance** $d_i$ as the distance between the image and the mirror along the principle axis.
Further, we need the **object height** $h_o$ and the **image height** $h_i$ to put everything into relation.
From the ratio of the heights and distances (similar triangle), respectively, we can derive the **mirror equation**:
$$\frac{h_0}{h_i}=\frac{d_0}{d_i}$$
Due to similar triangles in our ray diagram we see that the heights relate to the distance $d_0 -f$ and $f$:
$$\frac{h_0}{h_i}=\frac{d_0-f}{f}=\frac{d_0}{d_i}$$
By rearranging, we obtain the final **mirror equation** which relates the distances of object and image via the focal length ($f=\frac{r}{2}$):
$$\frac{1}{d_0} + \frac{1}{d_i} = \frac{1}{f}$$

Note that for an infinitely far away object, i.e. $d_0 = \infty$, we obtain $d_i = f$, as expected. 
Further, for a plane mirror, i.e. $f=\frac{r}{2}=\infty$, we obtain $d_i = -d_o$ (virtual image at same distance but behind mirror).

#### Magnification
**Lateral magnification** is defined as:
$$m=\frac{h_i}{h_o}=-\frac{d_i}{d_0}$$

**sign convention:**
* object height $h_o$ is always *positive*
* image height $h_i$ is *positive* if the image is *upright*; it is *negative* if the image is inverted
* distances, i.e. $d_i$ &amp; $d_o$, are *positive* if *in front of mirror*, *negative* if *behind mirror*

For example, a magnification $m\geq1$ means that the image is at least as big as the object and upright.

**Angular magnification** reflects better our perception of magnification in daily life as it compares two images instead of the image to the object (like lateral magnification):
$$M=\frac{\theta_C}{\theta_P}$$
In essence, in this context, it compares the apparent size of the image formed by a convex/concave mirror (suffix $C$) with the apparent size of the object as seen in a plane mirror (suffix $P$)  when the object is at the same distance from both mirrors.
For the same object, the angles are the angles subtended by the respective views at the observer's eye. The angles $\theta_C$ &amp; $\theta_P$ describe the apparent size of the object as seen through the curved mirror and the plane mirror, respectively.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Checkbox, Dropdown

# Constants
MIRROR_X = 0  # Mirror vertex at x = 0
OBJ_Y = -2  # Fixed object height
X_MAX = 10
Y_MAX = 5
MIN_RADIUS, MAX_RADIUS = 0.6 * X_MAX, X_MAX
MIN_OBJ_DIST, MAX_OBJ_DIST = 1, X_MAX

def compute_angle(x1, y1, x2, y2):
    """Compute the angle of a line segment from (x1, y1) to (x2, y2) with respect to the positive x-axis."""
    return np.arctan2(y2 - y1, x2 - x1)

def solve_quadratic(a, b, c):
    """Solve the quadratic equation ax^2 + bx + c = 0."""
    discriminant = b**2 - 4 * a * c
    if discriminant < 0:
        return None, None
    elif discriminant == 0:
        return (-b / (2 * a),) * 2
    else:
        return (-b + np.sqrt(discriminant)) / (2 * a), (-b - np.sqrt(discriminant)) / (2 * a)

def trace_ray(x0, y0, angle, mirror_radius, mirror_type):
    """Find the intersection of a ray with the spherical mirror and compute reflection."""
    if mirror_type not in ['concave', 'convex']:
        raise ValueError("Invalid mirror_type. Must be 'concave' or 'convex'.")

    # Determine mirror center based on type
    center_x = -mirror_radius if mirror_type == 'concave' else mirror_radius
    center_y = 0

    # Ray equation: y = m*x + c
    slope = np.tan(angle)
    intercept = y0 - slope * x0

    # Solve for intersection with the mirror circle: (x - center_x)^2 + y^2 = radius^2
    # Substitute y = slope*x + intercept: (x - center_x)^2 + (slope*x + intercept)^2 = radius^2
    # Expand and rearrange into a quadratic equation: A*x^2 + B*x + C = 0
    A = 1 + slope**2
    B = -2 * center_x + 2 * slope * intercept
    C = center_x**2 + intercept**2 - mirror_radius**2

    x_intersect1, x_intersect2 = solve_quadratic(A, B, C)

    if x_intersect1 is None:
        return None, None, None  # No valid intersection

    # Determine the correct intersection point based on mirror type and ray direction
    if mirror_type == 'concave':
        # For concave, the relevant intersection is the one on the right side of the center
        mirror_hit_x = max(x_intersect1, x_intersect2)
    elif mirror_type == 'convex':
        # For convex, the relevant intersection is the one on the left side of the center
        mirror_hit_x = min(x_intersect1, x_intersect2)

    mirror_hit_y = slope * mirror_hit_x + intercept

    # Compute normal angle at the point of intersection
    normal_angle = np.arctan2(mirror_hit_y - center_y, mirror_hit_x - center_x)

    # Compute incident angle (angle between incident ray and the normal)
    incident_angle = angle - normal_angle

    # Compute reflected angle (angle between reflected ray and the normal, equal in magnitude but opposite in sign to the incident angle)
    reflected_angle = normal_angle - incident_angle

    return mirror_hit_x, mirror_hit_y, reflected_angle

def plot_arrow(ax, x_start, y_start, x_end, y_end, color, head_width=0.25, head_length=0.3, alpha=1):
    """Helper function to plot an arrow at the midpoint of a ray."""
    mid_x = (x_start + x_end) / 2
    mid_y = (y_start + y_end) / 2
    dx = (x_end - x_start) * 0.2  # Scale arrow size
    dy = (y_end - y_start) * 0.2
    ax.arrow(mid_x, mid_y, dx, dy, head_width=head_width, head_length=head_length, fc=color, ec=color, alpha=alpha)

def find_line_intersection(x1, y1, angle1, x2, y2, angle2):
    """Compute the intersection point of two lines given a point and an angle for each."""
    if np.isclose(angle1 % np.pi, angle2 % np.pi):  # Check if angles are effectively parallel
        return None  # Lines are parallel

    slope1 = np.tan(angle1)
    slope2 = np.tan(angle2)

    intercept1 = y1 - slope1 * x1
    intercept2 = y2 - slope2 * x2

    intersection_x = (intercept2 - intercept1) / (slope1 - slope2)
    intersection_y = slope1 * intersection_x + intercept1
    return intersection_x, intersection_y

def plot_mirror_arc(ax, mirror_radius, mirror_type, num_points=100):
    """Plots the spherical mirror arc."""
    theta = np.linspace(-np.pi / 3, np.pi / 3, num_points)
    if mirror_type == 'concave':
        center_x = -mirror_radius
        arc_x = mirror_radius * np.cos(theta) + center_x
        arc_y = mirror_radius * np.sin(theta)
    elif mirror_type == 'convex':
        center_x = mirror_radius
        arc_x = -mirror_radius * np.cos(theta) + center_x
        arc_y = mirror_radius * np.sin(theta)
    else:
        raise ValueError("Invalid mirror_type")
    ax.plot(arc_x, arc_y, color='k', linewidth=3)

def plot_image_formation(object_distance, object_height, mirror_radius, show_central_ray, mirror_type):
    """Plots the object, mirror, principal rays, and the formed image."""
    if mirror_radius <= 0:
        return  # Avoid division by zero or invalid radius

    focal_length = abs(mirror_radius) / 2
    object_x = -object_distance

    # Determine focal point and center of curvature based on mirror type
    if mirror_type == 'concave':
        focal_point_x = -focal_length
        center_x = -mirror_radius
    elif mirror_type == 'convex':
        focal_point_x = focal_length
        center_x = mirror_radius
    else:
        raise ValueError("Invalid mirror_type")

    # Initialize plot
    fig, ax = plt.subplots(figsize=(8, 8))  # Increased figure size for better visualization

    # Plot principal axis, focal point, and center of curvature
    ax.plot([-X_MAX, X_MAX], [0, 0], color='gray', linestyle='dotted', label='Principal Axis')
    ax.plot([focal_point_x], [0], 'ro', label='Focal Point (F)')
    ax.text(focal_point_x + 0.2, 0.2, 'F', color='r', fontsize=12)
    ax.plot([center_x], [0], 'bo', label='Center of Curvature (C)')
    ax.text(center_x + 0.2, 0.2, 'C', color='b', fontsize=12)

    # Plot object as a thick vertical arrow
    arrow_width = 0.4
    arrow_head_length = 0.6
    ax.arrow(object_x, 0, 0, object_height, head_width=arrow_width, head_length=arrow_head_length,
             fc='k', ec='k', linewidth=2, length_includes_head=True, label='Object')
    ax.text(object_x - 0.5, np.sign(object_height) * (abs(object_height) + 0.5), 'Object', color='k', ha='center')


    # Plot mirror
    plot_mirror_arc(ax, mirror_radius, mirror_type)

    # Define principal Rays
    if mirror_type == 'concave':
        ray_definitions = [
            {"label": "Parallel Ray", "start": (object_x, object_height), "angle": 0, "color": 'g'},
            {"label": "Focal Ray", "start": (object_x, object_height),
             "angle_func": lambda: compute_angle(object_x, object_height, focal_point_x, 0), "color": 'r'},
            {"label": "Central Ray", "start": (object_x, object_height),
             "angle_func": lambda: compute_angle(object_x, object_height, center_x, 0), "color": 'b',
             "show": show_central_ray}
        ]
    elif mirror_type == 'convex':
        ray_definitions = [
            {"label": "Parallel Ray", "start": (object_x, object_height), "angle": 0, "color": 'g'},
            {"label": "Ray Towards Focus", "start": (object_x, object_height),
             "angle_func": lambda: compute_angle(object_x, object_height, focal_point_x, 0), "color": 'r'},
            {"label": "Ray Towards Center", "start": (object_x, object_height),
             "angle_func": lambda: compute_angle(object_x, object_height, center_x, 0), "color": 'b',
             "show": show_central_ray}
        ]

    reflected_ray_endpoints = []
    incident_ray_endpoints = []

    for ray_def in ray_definitions:
        if "show" in ray_def and not ray_def["show"]:
            continue

        start_x, start_y = ray_def["start"]
        angle = ray_def.get("angle")
        if "angle_func" in ray_def:
            angle = ray_def["angle_func"]()
        color = ray_def["color"]

        hit_x, hit_y, reflected_angle = trace_ray(start_x, start_y, angle, mirror_radius, mirror_type)

        if hit_x is not None:
            # Plot incident ray
            ax.plot([start_x, hit_x], [start_y, hit_y], color=color, linestyle='-', alpha=0.7)
            plot_arrow(ax, start_x, start_y, hit_x, hit_y, color, alpha=0.7)
            incident_ray_endpoints.append(((start_x, start_y), (hit_x, hit_y), color))

            # Compute reflected ray end point
            reflected_x_end = -X_MAX  # always extend to the left
            reflected_y_end = hit_y + np.tan(reflected_angle) * (reflected_x_end - hit_x)

            # Plot reflected ray
            ax.plot([hit_x, reflected_x_end], [hit_y, reflected_y_end], color=color, linestyle='-', alpha=0.7)
            plot_arrow(ax, hit_x, hit_y, reflected_x_end, reflected_y_end, color, alpha=0.7)
            reflected_ray_endpoints.append(((hit_x, hit_y), (reflected_x_end, reflected_y_end), color))

            # Plot extension of the ray (dashed) to find virtual image
            extended_x_end = X_MAX  # always extend to the right
            if mirror_type == 'convex':
                extended_y_end = hit_y + np.tan(reflected_angle) * (extended_x_end - hit_x)
                ax.plot([hit_x, extended_x_end], [hit_y, extended_y_end], color=color, linestyle='--', alpha=0.5)
            elif mirror_type == 'concave':
                # For concave, extensions might meet behind the mirror
                extended_y_end = hit_y + np.tan(reflected_angle) * (extended_x_end - hit_x)
                ax.plot([hit_x, extended_x_end], [hit_y, extended_y_end], color=color, linestyle='--', alpha=0.5)

    # Calculate and plot the intersection of the reflected rays (or their extensions)
    # We'll try to find the intersection of the first two reflected rays (parallel and focal/towards focus)
    if len(reflected_ray_endpoints) >= 2:
        (x1_start, y1_start), (x1_end, y1_end), _ = reflected_ray_endpoints[0]
        angle1 = compute_angle(x1_start, y1_start, x1_end, y1_end)

        (x2_start, y2_start), (x2_end, y2_end), _ = reflected_ray_endpoints[1]
        angle2 = compute_angle(x2_start, y2_start, x2_end, y2_end)

        intersection = find_line_intersection(x1_start, y1_start, angle1, x2_start, y2_start, angle2)
        if intersection:
            intersection_x, intersection_y = intersection
            # Check if the intersection point is within the boundaries
            if -X_MAX <= intersection_x <= X_MAX and -Y_MAX <= intersection_y <= Y_MAX:
                # Plot image as a thick vertical arrow
                ax.arrow(intersection_x, 0, 0, intersection_y, head_width=arrow_width, head_length=arrow_head_length,
                         fc='magenta', ec='magenta', linewidth=2, length_includes_head=True, label='Image')
                ax.text(intersection_x - 0.5, np.sign(intersection_y) * (abs(intersection_y) + 0.5), 'Image', color='magenta', ha='center')
            else:
                plt.legend(["Calculated image is outside the plotting boundaries."])
        else:
            print("Reflected rays (or their extensions) do not intersect within the plotting boundaries.")

    ax.set_xlim(-X_MAX, X_MAX)
    ax.set_ylim(-Y_MAX, Y_MAX)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_aspect('equal')
    ax.set_title(f"Image formation by a {mirror_type} mirror")
    plt.show()

interact(plot_image_formation,
         object_distance=FloatSlider(value=0.9*MAX_OBJ_DIST, min=MIN_OBJ_DIST, max=MAX_OBJ_DIST, step=0.1, description='Object Distance'),
         object_height=FloatSlider(value=1, min=-0.9 * Y_MAX, max=0.9 * Y_MAX, step=0.1, description='Object Height'),
         mirror_radius=FloatSlider(value=0.8*MAX_RADIUS, min=MIN_RADIUS, max=MAX_RADIUS, step=0.5, description='Mirror Radius'),
         show_central_ray=Checkbox(value=False, description='Show Central Ray'),
         mirror_type=Dropdown(options=['concave', 'convex'], value='concave', description='Mirror Type')
         );

interactive(children=(FloatSlider(value=9.0, description='Object Distance', max=10.0, min=1.0), FloatSlider(va…

## 2.9.5 Refraction &amp; Snell's law

The speed of light is $c\approx 300 \times 10^6 m/s$ in vacuum (virtually the same for air).
If light is traveling though other (transparent) materials, its speed is decreased (e.g. in water $\approx \frac{3}{4}c$) due to absorption and re-emission of light by the atoms in the material.
The **index of refraction** $n$ is defined as:
$$n = \frac{c}{v}$$

Typical values are: $n=1.33$ and $n=1.46$ for water and glass, respectively.

If light travels from one transparent medium to another one with a different index of refraction, **reflection** and **refraction** occur at the boundary.
While a certain portion of the light is reflected (see section before), due transparency, a fraction of the light is **transmitted** into the other medium.
However, due to the different index of refraction, the light's direction changes.
This "bending" of the light towards or away from the normal (w.r.t. boundary surface), is described by **Snell's law** (Willebrord Snell, 1591 - 1626)
$$ n_1 \sin \theta_1 = n_2 \sin \theta_2 $$

with $\theta_1$ as the angle of incidence, $\theta_2$ as the angle of refraction, and $n_1$ &amp; $n_2$ as the respective index of refraction.
Form the **law of refraction**, we conclude that bending towards the normal occurs if $n_2 > n_1$, i.e. the speed of light is slower in the second medium.
Refraction is the reason why we perceive optical illusions at for example the air-water-surface.



## 2.9.6 Total reflection

An incident ray can be refracted and reflected at the boundary between two transparent mediums, there are scenarios in which no transmission into the second medium occurs.
According to Snell's law, the angle of refraction depends on the two indexes of refraction and the incident angle.
If the angle of refraction is at least 90 degrees, no light is transmitted into the other medium.
This is the so-called **critical angle** $\theta_c$ and it can be simply derived from Snell's law as:
$$ n_1 \sin \theta_c = n_2 90\textrm{°}$$
$$ \sin \theta_c = \frac{n_2}{n_1}$$

If the angle of refraction is greater than 90 degrees, **total internal reflection** occurs.
This can only occur if light travels from a medium with higher to a medium with lower refraction index ($n_1 > n_2$).


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Global variable for ray length
RAY_LENGTH = 3

def snells_law(n1, n2, theta_incident):
    """Calculate the refraction angle using Snell's Law. Returns None if total internal reflection occurs."""
    theta_incident_rad = np.radians(theta_incident)
    sin_theta_refracted = (n1 / n2) * np.sin(theta_incident_rad)
    
    if abs(sin_theta_refracted) > 1:
        return None  # Total internal reflection
    
    theta_refracted_rad = np.arcsin(sin_theta_refracted)
    return np.degrees(theta_refracted_rad)

def get_medium_color(n):
    """Map refractive index to a color from white (n=1) to light blue (n=2) with max 70% opacity."""
    blue_intensity = (n - 1) / (2 - 1)
    return (1 - blue_intensity, 1 - blue_intensity, 1, 0.7 * blue_intensity)  # RGB with alpha

def compute_ray_coordinates(x_start, y_start, angle, length):
    """Compute the end coordinates of a ray given a start point, angle, and length."""
    x_end = x_start + length * np.cos(np.radians(angle))
    y_end = y_start + length * np.sin(np.radians(angle))
    return np.array([x_start, x_end]), np.array([y_start, y_end])

def plot_refraction(n1, n2, theta_incident):
    """Plot the refraction of a light beam through two media, handling total internal reflection."""
    theta_refracted = snells_law(n1, n2, theta_incident)
    
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # Plot mediums with color gradient
    ax.fill_between([-2, 2], -2, 0, color=get_medium_color(n2))  # Bottom medium
    ax.fill_between([-2, 2], 0, 2, color=get_medium_color(n1))   # Top medium
    
    ax.axhline(0, color='black', linewidth=2)  # Interface line
    
    # Compute and plot incident ray
    x_incident, y_incident = compute_ray_coordinates(0, 0, 90-theta_incident, RAY_LENGTH)
    ax.plot(x_incident, y_incident, 'b', linewidth=2.5, label='Incident Ray')

    if theta_refracted is None:
        # Total internal reflection: Compute and plot reflected ray
        theta_reflected = theta_incident
        x_reflected, y_reflected = compute_ray_coordinates(0, 0, 90 + theta_reflected, RAY_LENGTH)
        ax.plot(x_reflected, y_reflected, 'g', linewidth=2.5, label='Reflected Ray')
        label_refracted = "total internal\nreflection"
    else:
        # Compute and plot refracted ray
        x_refracted, y_refracted = compute_ray_coordinates(0, 0, 270-theta_refracted, RAY_LENGTH)
        ax.plot(x_refracted, y_refracted, 'r', linewidth=2.5, label='Refracted Ray')
        label_refracted = f"n₂ = {n2:.2f}\nθ₂ = {theta_refracted:.1f}°"
    
    # Normal line
    ax.plot([0, 0], [-2, 2], 'k--', linewidth=1.5, label='Normal')
    
    # Labels
    ax.text(-1, 1.2, f"n₁ = {n1:.2f}\nθ₁ = {theta_incident:.1f}°", color='black', fontsize=12)
    ax.text(0.5, -0.5, label_refracted, color='black', fontsize=12)
    
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.legend()
    ax.set_aspect('equal')
    ax.set_title("Refraction incl. total internal reflection")
    plt.show()

# Interactive widgets
interact(plot_refraction, 
         n1=FloatSlider(value=1.0, min=1.0, max=2.0, step=0.01, description='n1'),
         n2=FloatSlider(value=1.5, min=1.0, max=2.0, step=0.01, description='n2'),
         theta_incident=FloatSlider(value=45, min=1, max=89, step=1, description='θ₁'))


interactive(children=(FloatSlider(value=1.0, description='n1', max=2.0, min=1.0, step=0.01), FloatSlider(value…

<function __main__.plot_refraction(n1, n2, theta_incident)>

## 2.9.7 Dispersion and the visible spectrum

The index of refraction $n$ is actually depending on the wavelength $\lambda$.
Thus, depending on the wave length, the degree of refraction changes.
This is the reason why we see rainbows, but let's start more general:
Light can have different **color** and **intensity**.
While the intensity, i.e. brightness, depends on the energy per unit area and unit time, the color of light is represented by its wavelength (or frequency).

In the previous chapter we defined the relation of wavelength and frequency via the speed of light *in vacuum*, i.e. $\lambda=\frac{c}{f}$.
Let's be more general and define the wavelength of an electromagnetic wave in a medium as $\lambda_n$.
The wavelength in the medium will be the speed of light within that medium $v$ divided by frequency $f$, which gives us the following insight:
$$\lambda_n = \frac{v}{f} = \frac{c}{nf} = \frac{\lambda}{n}$$

Thus, the frequency is independent of the medium light travels through (because as the wave travels though two different mediums, atoms next to each other will vibrate with the same/similar frequency, even at boundaries; no abrupt changes in frequency).
Most likely, that is the reason why our brain's can interpret "red" as "red" in air and underwater (cells in the eye's retina being frequency and not wavelength sensitive). 

As while light is the superposition of all different wavelengths, a simple triangle of glass, a so-called prism, can be used to decompose white light into its components.
This wavelength-dependent refraction of white light into is **spectrum** is called **dispersion**.
Visible light is only a small part of the electromagnetic spectrum. 
In air, violet light has a wavelength of $400$ nm ($f \approx 7.5 \times 10^{14}$ Hz) and wavelengths shorter that that are from the so-called **ultraviolet** (UV) spectrum.
Red light has a wavelength of $700$ nm ($f \approx 4.3 \times 10^{14}$ Hz) and wavelengths longer that that are from the so-called **infrared** (IR) spectrum.
